# Run the CICI SAFE vSDX

#### Configure the Envrionment

In [1]:
# Set up user's project (user's can be multiple ones, so there is no default currently)
export OS_PROJECT_NAME='CH-816532'

# Set region (again, no default currently)
export OS_REGION_NAME='CHI@TACC'

# Set chameleon keypair name and path to the private ssh key
export SSH_KEY_NAME=${USERNAME}'-jupyter'
export SSH_PRIVATE_KEY=${HOME}/.ssh/id_rsa

# Set the reservations to use.  
# CONTROLLER_RESERVATION_ID can be for any type of node
export CLIENT_RESERVATION_ID='86df92e0-3608-48e1-83cb-c27b64fa2083'

#Set the working directory
export WORKING_DIR=${HOME}/work/CICI-SAFE/Chameleon-Jupyter/ExoGENI-Client

#Set GENI CERT file
export GENI_CERT=${HOME}/work/geni-pruth.pem

export SDX_PUBLIC_IP='129.114.108.123'

#### Start ExoGENI Client

In [2]:
cd $WORKING_DIR
CLIENT_NETWORK='sharednet1'
CLIENT_NODE_NAME=${USERNAME}-client2
CLIENT_STACK_NAME=${USERNAME}-client2
GIT_BRANCH='tridentcom'
SLICE_NAME='c1-tri'
IP_PREFIX='192.168.20.1/24'
CLIENT_SITE='UFL'
SAFE_KEYPAIR='key_p11'

echo Creating SDX client. This will take several minutes!
openstack stack create  --max-width 80 \
        --template ${WORKING_DIR}/client.yaml \
        --parameter "key_name=${SSH_KEY_NAME}" \
        --parameter "reservation_id=${CLIENT_RESERVATION_ID}" \
        --parameter "network_name=${CLIENT_NETWORK}" \
        --parameter "client_name=${CLIENT_NODE_NAME}" \
        --parameter "github_branch=${GIT_BRANCH}" \
        --parameter "riak_ip=${RIAK_PUBLIC_IP}" \
        --parameter "slice_name=${SLICE_NAME}"\
        --parameter "ip_prefix=${IP_PREFIX}"\
        --parameter "client_site=${CLIENT_SITE}"\
        --parameter "safe_keypair=${SAFE_KEYPAIR}"\
        --parameter "sdx_server=${SDX_PUBLIC_IP}:8888"\
        --wait \
        ${CLIENT_STACK_NAME}

Creating SDX client. This will take several minutes!
2019-02-22 15:44:18Z [pruth-client2]: CREATE_IN_PROGRESS  Stack CREATE started
2019-02-22 15:44:19Z [pruth-client2.sdx_floating_ip]: CREATE_IN_PROGRESS  state changed
2019-02-22 15:44:21Z [pruth-client2.sdx_floating_ip]: CREATE_COMPLETE  state changed
2019-02-22 15:44:22Z [pruth-client2.sdx-client]: CREATE_IN_PROGRESS  state changed
2019-02-22 15:53:33Z [pruth-client2.sdx-client]: CREATE_COMPLETE  state changed
2019-02-22 15:53:34Z [pruth-client2.sdx_ip_association]: CREATE_IN_PROGRESS  state changed
2019-02-22 15:53:36Z [pruth-client2.sdx_ip_association]: CREATE_COMPLETE  state changed
2019-02-22 15:53:36Z [pruth-client2]: CREATE_COMPLETE  Stack CREATE completed successfully
+---------------------+--------------------------------------------------------+
| Field               | Value                                                  |
+---------------------+--------------------------------------------------------+
| id               

In [8]:
echo SDX client creation complete! 
echo ${CLIENT_NODE_NAME} status `openstack server show  --format value -c status ${CLIENT_NODE_NAME}`

CLIENT_PUBLIC_IP=`openstack server show  --format value -c addresses ${CLIENT_NODE_NAME} | cut -d " " -f 2`
echo The client public IP is $CLIENT_PUBLIC_IP


#Poll for node to be ready for ssh
while [[ true ]]
do 
	ssh -q -i ${SSH_PRIVATE_KEY} \
    -o UserKnownHostsFile=/dev/null \
    -o StrictHostKeyChecking=no \
    cc@${CLIENT_PUBLIC_IP} \
    exit
    
    if [[ "$?" == "0" ]]; then break; fi
    
    echo Node not ready for ssh. retrying in 5 seconds.
    sleep 5
done

echo SDX node ready for ssh

SDX client creation complete!
pruth-client1 status ACTIVE
The client public IP is 129.114.108.64
SDX node ready for ssh


In [9]:
#Copy GENI pem file to node
REMOTE_GENI_CERT=/home/cc/geni.pem

echo Copying GENI cert to cc@$CLIENT_PUBLIC_IP:~/geni.pem
scp -q -i ${SSH_PRIVATE_KEY} \
    -o UserKnownHostsFile=/dev/null \
    -o StrictHostKeyChecking=no \
    ${GENI_CERT} \
    cc@${CLIENT_PUBLIC_IP}:${REMOTE_GENI_CERT}
echo Done

Copying GENI cert to cc@129.114.108.64:~/geni.pem
Done


In [ ]:
#Tail log until complete. Note: YOU MUST MANUALLY STOP THIS TO CONTINUE!
ssh -i ${SSH_PRIVATE_KEY} \
    -o UserKnownHostsFile=/dev/null \
    -o StrictHostKeyChecking=no \
    cc@${CLIENT_PUBLIC_IP} \
    sudo tail -F /home/cc/boot.log

Downloading: http://ci-dev.renci.org/nexus/content/repositories/geni-orca-libs/org/codehaus/plexus/plexus/1.0.10/plexus-1.0.10.pom
Downloading: http://repository.apache.org/snapshots/org/codehaus/plexus/plexus/1.0.10/plexus-1.0.10.pom
Downloading: https://repo.maven.apache.org/maven2/org/codehaus/plexus/plexus/1.0.10/plexus-1.0.10.pom
Downloaded: https://repo.maven.apache.org/maven2/org/codehaus/plexus/plexus/1.0.10/plexus-1.0.10.pom (9 KB at 383.6 KB/sec)
Downloading: http://ci-dev.renci.org/nexus/content/repositories/geni-orca-libs/org/codehaus/plexus/plexus-container-default/1.0-alpha-15/plexus-container-default-1.0-alpha-15.pom
Downloading: http://repository.apache.org/snapshots/org/codehaus/plexus/plexus-container-default/1.0-alpha-15/plexus-container-default-1.0-alpha-15.pom
Downloading: https://repo.maven.apache.org/maven2/org/codehaus/plexus/plexus-container-default/1.0-alpha-15/plexus-container-default-1.0-alpha-15.pom
Downloaded: https://repo.maven.apache.org/maven2/org/codeh

#### Start ExoGENI Client 2

In [ ]:
cd $WORKING_DIR
CLIENT_NETWORK='sharednet1'
CLIENT_NODE_NAME2=${USERNAME}-client2
CLIENT_STACK_NAME2=${USERNAME}-clien2
GIT_BRANCH='tridentcom'
SLICE_NAME2='c1-tri'
IP_PREFIX2='192.168.20.1/24'
CLIENT_SITE2='UFL'
SAFE_KEYPAIR2='key_p11'

echo Creating SDX client. This will take several minutes!
openstack stack create  --max-width 80 \
        --template client.yaml \
        --parameter "key_name=${SSH_KEY_NAME}" \
        --parameter "reservation_id=${CLIENT_RESERVATION_ID}" \
        --parameter "network_name=${CLIENT_NETWORK}" \
        --parameter "client_name=${CLIENT_NODE_NAME2}" \
        --parameter "github_branch=${GIT_BRANCH}" \
        --parameter "riak_ip=${RIAK_PUBLIC_IP2}" \
        --parameter "slice_name=${SLICE_NAME2}"\
        --parameter "ip_prefix=${IP_PREFIX2}"\
        --parameter "client_site=${CLIENT_SITE2}"\
        --parameter "safe_keypair=${SAFE_KEYPAIR2}"\
        --parameter "sdx_server=${SDX_PUBLIC_IP}:8888"\
        --wait \
        ${CLIENT_STACK_NAME2}

In [ ]:
echo SDX client creation complete! 
echo ${CLIENT_NODE_NAME2} status `openstack server show  --format value -c status ${CLIENT_NODE_NAME2}`

CLIENT_PUBLIC_IP2=`openstack server show  --format value -c addresses ${CLIENT_NODE_NAME2} | cut -d " " -f 2`
echo The client public IP is $CLIENT_PUBLIC_IP2

result=255
while [[ "$result" != "0" ]]
do 
    sleep 5
	ssh -q -i ${SSH_PRIVATE_KEY} \
    -o UserKnownHostsFile=/dev/null \
    -o StrictHostKeyChecking=no \
    cc@${CLIENT_PUBLIC_IP2} \
    exit
	result=$?
done

echo COPY your geni pem to cc@$CLIENT_PUBLIC_IP2:~/geni.pem

ssh -i ${SSH_PRIVATE_KEY} \
    -o UserKnownHostsFile=/dev/null \
    -o StrictHostKeyChecking=no \
    cc@${CLIENT_PUBLIC_IP2} \
    sudo tail /root/boot.log


#### Client 1 stitch to SDX

In [ ]:
ssh -i ${SSH_PRIVATE_KEY} \
    -o UserKnownHostsFile=/dev/null \
    -o StrictHostKeyChecking=no \
    cc@${CLIENT_PUBLIC_IP1} \
    "cd ~/CICI-SAFE/SDX-Simple &&
    sudo ./scripts/sdx_exogeni_client.sh -c ~/client.conf -e 'stitch CNode1 192.168.10.2 192.168.10.1/24'"

#### Client 1 advertise its IP prefix

In [ ]:
ssh -i ${SSH_PRIVATE_KEY} \
    -o UserKnownHostsFile=/dev/null \
    -o StrictHostKeyChecking=no \
    cc@${CLIENT_PUBLIC_IP1} \
    "cd ~/CICI-SAFE/SDX-Simple && pwd&&
    sudo ./scripts/sdx_exogeni_client.sh -c ~/client.conf -e 'route 192.168.10.1/24 192.168.10.2'"

#### Client 2 stitch to SDX

In [ ]:
ssh -i ${SSH_PRIVATE_KEY} \
    -o UserKnownHostsFile=/dev/null \
    -o StrictHostKeyChecking=no \
    cc@${CLIENT_PUBLIC_IP2} \
    "cd ~/CICI-SAFE/SDX-Simple && pwd&&
    sudo ./scripts/sdx_exogeni_client.sh -c ~/client.conf -e 'stitch CNode1 192.168.20.2 192.168.20.1/24'"

#### Client 2 advertise its IP prefix

In [ ]:
ssh -i ${SSH_PRIVATE_KEY} \
    -o UserKnownHostsFile=/dev/null \
    -o StrictHostKeyChecking=no \
    cc@${CLIENT_PUBLIC_IP2} \
    "cd ~/CICI-SAFE/SDX-Simple && pwd&&
    sudo ./scripts/sdx_exogeni_client.sh -c ~/client.conf -e 'route 192.168.20.1/24 192.168.20.2'"

#### Connect client 1 and client 2

In [ ]:
ssh -i ${SSH_PRIVATE_KEY} \
    -o UserKnownHostsFile=/dev/null \
    -o StrictHostKeyChecking=no \
    cc@${CLIENT_PUBLIC_IP1} \
    "cd ~/CICI-SAFE/SDX-Simple && pwd&&
    sudo ./scripts/sdx_exogeni_client.sh -c ~/client.conf -e 'link 192.168.10.1/24 192.168.20.1/24'"

#### Undo stitching

In [ ]:
ssh -i ${SSH_PRIVATE_KEY} \
    -o UserKnownHostsFile=/dev/null \
    -o StrictHostKeyChecking=no \
    cc@${CLIENT_PUBLIC_IP1} \
    "cd ~/CICI-SAFE/SDX-Simple &&
    sudo ./scripts/sdx_exogeni_client.sh -c ~/client.conf -e 'unstitch CNode1'"

In [ ]:
ssh -i ${SSH_PRIVATE_KEY} \
    -o UserKnownHostsFile=/dev/null \
    -o StrictHostKeyChecking=no \
    cc@${CLIENT_PUBLIC_IP2} \
    "cd ~/CICI-SAFE/SDX-Simple &&
    sudo ./scripts/sdx_exogeni_client.sh -c ~/client.conf -e 'unstitch CNode1'"

# Please Cleanup Your Resources!!!

#### Delete SDX Stack

In [ ]:
echo Deleting stack ${SDX_STACK_NAME}
openstack stack delete -y --wait ${SDX_STACK_NAME}
echo Stack ${SDX_STACK_NAME} deleted

#### Delete RIAK Stack

In [ ]:
echo Deleting stack ${RIAK_STACK_NAME}
openstack stack delete -y --wait ${RIAK_STACK_NAME}
echo Stack ${RIAK_STACK_NAME} deleted

#### Delete Client Stack

In [ ]:
echo Deleting stack ${CLIENT_STACK_NAME1}
openstack stack delete -y --wait ${CLIENT_STACK_NAME1}
echo Stack ${CLIENT_STACK_NAME1} deleted

In [ ]:
echo Deleting stack ${CLIENT_STACK_NAME2}
openstack stack delete -y --wait ${CLIENT_STACK_NAME2}
echo Stack ${CLIENT_STACK_NAME2} deleted